# General DA fittings and Fokker-Planck comparisons (Hénon map)

## Setup scripts if we are under SWAN

In [ ]:
# Working in the right path
%cd /eos/project/d/da-and-diffusion-studies/DA_Studies/Simulations/Models/loss_studies/notebooks

In [ ]:
# Install the libraries
import sys
!{sys.executable} -m pip install --user tqdm pynverse sixtrackwrap crank-nicolson-numba henon-map symplectic-map
!{sys.executable} -m pip install --user --upgrade sixtrackwrap 
!{sys.executable} -m pip install --user --upgrade crank-nicolson-numba 
!{sys.executable} -m pip install --user --upgrade henon-map 
!{sys.executable} -m pip install --user --upgrade symplectic-map
!export PYTHONPATH=$CERNBOX_HOME.local/lib/python3.7/site-packages:$PYTHONPATH

In [ ]:
# For this "presentation" only!
import warnings
warnings.filterwarnings('ignore')

## Imports

In [2]:
%matplotlib widget

In [3]:
# Base libraries
import math
import numpy as np
import scipy.integrate as integrate
from scipy.special import erf
import pickle
import itertools
from scipy.optimize import curve_fit

from numba import njit, prange

from tqdm.notebook import tqdm
import time
import matplotlib.pyplot as plt
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.ticker as ticker
from math import gcd

import pandas as pd

from scipy.special import lambertw
from scipy.interpolate import interp1d

import os

# Personal libraries
import sixtrackwrap as sx
import crank_nicolson_numba.nekhoroshev as nk

# Personal modules
import fit_utils as fit

/home/carlidel/Insync/carlo.montanari3@studio.unibo.it/OneDrive Biz/projects/loss_studies/notebooks/fit_utils.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


## Load data and setup original DA

In [4]:
savepath = "../data/"

sigma = 0.3
turn_samples = 100

# Do we want to load ALL the Hénon data files or should we skip some?
skipper = 1

cut_point = 0.9

In [7]:
epsilons = []

real_losses = []
gaussian_losses = []

real_DAs = []
gaussian_DAs = []

files = list(sorted(list(filter(lambda f: "henon_eps_" in f, os.listdir(savepath))), key=lambda f: float(f[10: -4])))[::skipper]

for f in tqdm(files):
    print("Loading file: ", f)
    epsilon = float(f[10: -4])
    epsilons.append(epsilon)
    engine = sx.uniform_radial_scanner.load_values(savepath + f)

    baseline_samples = engine.baseline_samples
    min_turns = 10**3
    max_turns = 10**6
    d_r = engine.dr
    turn_sampling = np.linspace(min_turns, max_turns, turn_samples, dtype=np.int)[::-1]
    
    print("Real part")
    real_DAs.append(engine.compute_DA_standard(turn_sampling))
    print("Done DA")
    
    print("Gaussian part")
    engine.assign_weights(sx.assign_symmetric_gaussian(sigma))
    gaussian_losses.append(engine.compute_loss(turn_sampling, cut_point))
    gaussian_DAs.append(fit.DA_from_symmetric_gaussian_loss(gaussian_losses[-1], sigma, cut_point))
    print("Done Gaussian")
    
    print("Gaussian loss but with 'Real' DA values")
    baseline = engine.compute_loss_cut(cut_point)
    real_losses.append(np.array([engine.compute_loss_cut(da) / baseline for da in real_DAs[-1]]))
    print("Finished processing.")

Loading file:  henon_eps_2.pkl
Real part
Done DA
Gaussian part
Done Gaussian
Gaussian loss but with 'Real' DA values
Finished processing.
Loading file:  henon_eps_6.pkl
Real part
Done DA
Gaussian part
Done Gaussian
Gaussian loss but with 'Real' DA values
Finished processing.
Loading file:  henon_eps_10.pkl
Real part
Done DA
Gaussian part
Done Gaussian
Gaussian loss but with 'Real' DA values
Finished processing.
Loading file:  henon_eps_14.pkl
Real part
Done DA
Gaussian part
Done Gaussian
Gaussian loss but with 'Real' DA values
Finished processing.
Loading file:  henon_eps_18.pkl
Real part
Done DA
Gaussian part
Done Gaussian
Gaussian loss but with 'Real' DA values
Finished processing.
Loading file:  henon_eps_22.pkl
Real part
Done DA
Gaussian part
Done Gaussian
Gaussian loss but with 'Real' DA values
Finished processing.
Loading file:  henon_eps_26.pkl
Real part
Done DA
Gaussian part
Done Gaussian
Gaussian loss but with 'Real' DA values
Finished processing.
Loading file:  henon_eps_30.p

## General fitting of all the Hénon maps across all models

In [16]:
labels = (
    ("epsilon", ""),
    ("type", ""),
    ("sigma", ""),
    ("Model 2", "k"),
    ("Model 2", "k err"),
    ("Model 2", "rho"),
    ("Model 2", "rho err"),
    ("Model 2", "N0"),
    ("Model 2", "N0 err"),
    ("Model 2", "Chi2"),
    ("Model 4 (2 free pars)", "k"),
    ("Model 4 (2 free pars)", "k err"),
    ("Model 4 (2 free pars)", "rho"),
    ("Model 4 (2 free pars)", "rho err"),
    ("Model 4 (2 free pars)", "lambda"),
    ("Model 4 (2 free pars)", "Chi2"),
    ("Model 4 (3 free pars)", "k"),
    ("Model 4 (3 free pars)", "k err"),
    ("Model 4 (3 free pars)", "rho"),
    ("Model 4 (3 free pars)", "rho err"),
    ("Model 4 (3 free pars)", "N0"),
    ("Model 4 (3 free pars)", "N0 err"),
    ("Model 4 (3 free pars)", "lambda"),
    ("Model 4 (3 free pars)", "Chi2"),
)

fitting_data = pd.DataFrame(columns=pd.MultiIndex.from_tuples(labels))
lost_table = []

In [17]:
k_min = 0.1
k_max = 2.00
samples = 100

ks = np.linspace(k_min, k_max, samples)

In [18]:
for i, eps in tqdm(list(enumerate(epsilons))):
    # Model 2
    real_pars, real_errs, real_co_pars = fit.explore_k_model_2(
        turn_sampling, real_DAs[i], k_min, k_max, samples
    )
    gaussian_loss_pars, gaussian_loss_errs, gaussian_loss_co_pars = fit.explore_k_model_2(
        turn_sampling, gaussian_DAs[i], k_min, k_max, samples
    )

    real_selected_err_2 = np.min(real_errs)
    real_selected_k_2 = ks[np.argmin(real_errs)]
    real_selected_pars_2 = real_pars[np.argmin(real_errs)]
    real_selected_co_pars_2 = real_co_pars[np.argmin(real_errs)]

    gaussian_loss_selected_err_2 = np.min(gaussian_loss_errs)
    gaussian_loss_selected_k_2 = ks[np.argmin(gaussian_loss_errs)]
    gaussian_loss_selected_pars_2 = gaussian_loss_pars[np.argmin(gaussian_loss_errs)]
    gaussian_loss_selected_co_pars_2 = gaussian_loss_co_pars[np.argmin(gaussian_loss_errs)]
    
    # Model 4 (2 pars)
    real_pars, real_errs, real_co_pars = fit.explore_k_model_4(
        turn_sampling, real_DAs[i], k_min, k_max, samples
    )
    gaussian_loss_pars, gaussian_loss_errs, gaussian_loss_co_pars = fit.explore_k_model_4(
        turn_sampling, gaussian_DAs[i], k_min, k_max, samples
    )

    real_selected_err_4 = np.min(real_errs)
    real_selected_k_4 = ks[np.argmin(real_errs)]
    real_selected_pars_4 = real_pars[np.argmin(real_errs)]
    real_selected_co_pars_4 = real_co_pars[np.argmin(real_errs)]

    gaussian_loss_selected_err_4 = np.min(gaussian_loss_errs)
    gaussian_loss_selected_k_4 = ks[np.argmin(gaussian_loss_errs)]
    gaussian_loss_selected_pars_4 = gaussian_loss_pars[np.argmin(gaussian_loss_errs)]
    gaussian_loss_selected_co_pars_4 = gaussian_loss_co_pars[np.argmin(gaussian_loss_errs)]
    
    # Model 4 (3 pars)
    real_pars, real_errs, real_co_pars = fit.explore_model_4_free(
        turn_sampling, real_DAs[i], k_min, k_max, samples
    )
    gaussian_loss_pars, gaussian_loss_errs, gaussian_loss_co_pars = fit.explore_model_4_free(
        turn_sampling, gaussian_DAs[i], k_min, k_max, samples
    )

    real_selected_err_4_free = np.min(real_errs)
    real_selected_k_4_free = ks[np.argmin(real_errs)]
    real_selected_pars_4_free = real_pars[np.argmin(real_errs)]
    real_selected_co_pars_4_free = real_co_pars[np.argmin(real_errs)]

    gaussian_loss_selected_err_4_free = np.min(gaussian_loss_errs)
    gaussian_loss_selected_k_4_free = ks[np.argmin(gaussian_loss_errs)]
    gaussian_loss_selected_pars_4_free = gaussian_loss_pars[np.argmin(gaussian_loss_errs)]
    gaussian_loss_selected_co_pars_4_free = gaussian_loss_co_pars[np.argmin(gaussian_loss_errs)]
    
    # Saving Data
    fitting_data.loc[len(fitting_data)] = [
        epsilons[i],
        "real",
        np.nan,
        real_selected_k_2,
        ks[1] - ks[0],
        real_selected_pars_2[0],
        np.sqrt(real_selected_co_pars_2[0][0]),
        real_selected_pars_2[1],
        np.sqrt(real_selected_co_pars_2[1][1]),
        real_selected_err_2,
        real_selected_k_4,
        ks[1] - ks[0],
        real_selected_pars_4[0],
        np.sqrt(real_selected_co_pars_4[0][0]),
        1/2,
        real_selected_err_4,
        real_selected_k_4_free,
        ks[1] - ks[0],
        real_selected_pars_4_free[0],
        np.sqrt(real_selected_co_pars_4_free[0][0]),
        real_selected_pars_4_free[1],
        np.sqrt(real_selected_co_pars_4_free[1][1]),
        1/2,
        real_selected_err_4_free
    ]
    
    lost_table.append(real_losses[i])
    
    fitting_data.loc[len(fitting_data)] = [
        epsilons[i],
        "gaussian",
        sigma,
        gaussian_loss_selected_k_2,
        ks[1] - ks[0],
        gaussian_loss_selected_pars_2[0],
        np.sqrt(gaussian_loss_selected_co_pars_2[0][0]),
        gaussian_loss_selected_pars_2[1],
        np.sqrt(gaussian_loss_selected_co_pars_2[1][1]),
        gaussian_loss_selected_err_2,
        gaussian_loss_selected_k_4,
        ks[1] - ks[0],
        gaussian_loss_selected_pars_4[0],
        np.sqrt(gaussian_loss_selected_co_pars_4[0][0]),
        1/2,
        gaussian_loss_selected_err_4,
        gaussian_loss_selected_k_4_free,
        ks[1] - ks[0],
        gaussian_loss_selected_pars_4_free[0],
        np.sqrt(gaussian_loss_selected_co_pars_4_free[0][0]),
        gaussian_loss_selected_pars_4_free[1],
        np.sqrt(gaussian_loss_selected_co_pars_4_free[1][1]),
        1/2,
        gaussian_loss_selected_err_4_free
    ]
    
    lost_table.append(gaussian_losses[i])

### Visualize the $\kappa$ value for the various models

In [24]:
fig1, ax1 = plt.subplots(1,2)

p_filter = fitting_data[("type", "")] == "real" 

ax1[0].scatter(
    fitting_data[p_filter][("epsilon", "")],
    fitting_data[p_filter][("Model 2", "k")],
    label="Model 2"
)
ax1[0].scatter(
    fitting_data[p_filter][("epsilon", "")],
    fitting_data[p_filter][("Model 4 (2 free pars)", "k")],
    label="Model 4 (2 pars)"
)
ax1[0].scatter(
    fitting_data[p_filter][("epsilon", "")],
    fitting_data[p_filter][("Model 4 (3 free pars)", "k")],
    label="Model 4 (3 pars)"
)

ax1[0].set_xlabel("$\\epsilon$")
ax1[0].set_ylabel("$\\kappa$")
ax1[0].set_title("Real DA")
ax1[0].legend(fontsize="xx-small")

p_filter = fitting_data[("type", "")] == "gaussian" 

ax1[1].scatter(
    fitting_data[p_filter][("epsilon", "")],
    fitting_data[p_filter][("Model 2", "k")],
    label="Model 2"
)
ax1[1].scatter(
    fitting_data[p_filter][("epsilon", "")],
    fitting_data[p_filter][("Model 4 (2 free pars)", "k")],
    label="Model 4 (2 pars)"
)
ax1[1].scatter(
    fitting_data[p_filter][("epsilon", "")],
    fitting_data[p_filter][("Model 4 (3 free pars)", "k")],
    label="Model 4 (3 pars)"
)

ax1[1].set_xlabel("$\\epsilon$")
ax1[1].set_ylabel("$\\kappa$")
ax1[1].set_title("DA from Gaussian loss")
ax1[1].legend(fontsize="xx-small")

p_filter = fitting_data[("type", "")] == "uniform" 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
fitting_data[["epsilon", "type", "sigma", "Model 2"]]

epsilon      type sigma   Model 2                                 \
                                   k     k err        rho   rho err   
0      2.0      real   NaN  0.234343  0.019192   1.835630  0.001863   
1      2.0  gaussian   0.3  0.138384  0.019192   1.108686  0.000312   
2      6.0      real   NaN  0.368687  0.019192   3.509941  0.002510   
3      6.0  gaussian   0.3  0.253535  0.019192   1.946185  0.001527   
4     10.0      real   NaN  0.483838  0.019192   6.072436  0.004085   
5     10.0  gaussian   0.3  0.311111  0.019192   2.540155  0.001730   
6     14.0      real   NaN  0.656566  0.019192  13.928873  0.016458   
7     14.0  gaussian   0.3  0.349495  0.019192   3.021043  0.001685   
8     18.0      real   NaN  0.771717  0.019192  23.740494  0.036355   
9     18.0  gaussian   0.3  0.368687  0.019192   3.266681  0.001263   
10    22.0      real   NaN  0.867677  0.019192  36.195525  0.095800   
11    22.0  gaussian   0.3  0.387879  0.019192   3.539406  0.001287   
12    26.0      real   NaN  0.906061  0.019192  41.431612  0.116709   
13    26.0  gaussian   0.3  0.407071  0.019192   3.828161  0.001858   
14    30.0      real   NaN  0.944444  0.019192  47.853141  0.125729   
15    30.0  gaussian   0.3  0.464646  0.019192   4.982714  0.004215   

                                        
            N0        N0 err      Chi2  
0     3.913099  1.875598e-01  0.000110  
1   174.072875  2.346540e+00  0.000057  
2     0.185349  5.104695e-03  0.000026  
3    12.686566  3.839377e-01  0.000103  
4     0.015448  3.590381e-04  0.000015  
5     4.402483  1.050076e-01  0.000054  
6     0.000313  1.163130e-05  0.000028  
7     2.304382  4.254840e-02  0.000030  
8     0.000037  1.665938e-06  0.000038  
9     1.937657  2.387312e-02  0.000014  
10    0.000010  7.667986e-07  0.000103  
11    1.551573  1.740717e-02  0.000012  
12    0.000013  9.422083e-07  0.000117  
13    1.291900  1.872808e-02  0.000021  
14    0.000012  7.891996e-07  0.000099  
15    0.450558  1.085980e-02  0.000048

In [21]:
fitting_data[["epsilon", "type", "sigma", "Model 4 (2 free pars)"]]

epsilon      type sigma Model 4 (2 free pars)                      \
                                               k     k err       rho   
0      2.0      real   NaN              0.291919  0.019192  2.528580   
1      2.0  gaussian   0.3              0.311111  0.019192  2.800042   
2      6.0      real   NaN              0.349495  0.019192  3.203132   
3      6.0  gaussian   0.3              0.368687  0.019192  3.556280   
4     10.0      real   NaN              0.387879  0.019192  3.729218   
5     10.0  gaussian   0.3              0.407071  0.019192  4.146752   
6     14.0      real   NaN              0.407071  0.019192  3.999251   
7     14.0  gaussian   0.3              0.426263  0.019192  4.459445   
8     18.0      real   NaN              0.445455  0.019192  4.648181   
9     18.0  gaussian   0.3              0.426263  0.019192  4.426356   
10    22.0      real   NaN              0.464646  0.019192  4.977479   
11    22.0  gaussian   0.3              0.445455  0.019192  4.763605   
12    26.0      real   NaN              0.483838  0.019192  5.325588   
13    26.0  gaussian   0.3              0.464646  0.019192  5.121889   
14    30.0      real   NaN              0.503030  0.019192  5.698385   
15    30.0  gaussian   0.3              0.483838  0.019192  5.499997   

                               
     rho err lambda      Chi2  
0   0.000488    0.5  0.000167  
1   0.001302    0.5  0.000963  
2   0.000280    0.5  0.000036  
3   0.000883    0.5  0.000269  
4   0.000384    0.5  0.000048  
5   0.000676    0.5  0.000120  
6   0.000806    0.5  0.000194  
7   0.000557    0.5  0.000071  
8   0.001130    0.5  0.000251  
9   0.000606    0.5  0.000074  
10  0.001708    0.5  0.000521  
11  0.000431    0.5  0.000033  
12  0.002068    0.5  0.000665  
13  0.000448    0.5  0.000034  
14  0.002294    0.5  0.000697  
15  0.000654    0.5  0.000065

In [22]:
fitting_data[["epsilon", "type", "sigma", "Model 4 (3 free pars)"]]

epsilon      type sigma Model 4 (3 free pars)                       \
                                               k     k err        rho   
0      2.0      real   NaN              0.234343  0.019192   1.841246   
1      2.0  gaussian   0.3              0.157576  0.019192   1.228276   
2      6.0      real   NaN              0.368687  0.019192   3.529348   
3      6.0  gaussian   0.3              0.253535  0.019192   1.954448   
4     10.0      real   NaN              0.483838  0.019192   6.120074   
5     10.0  gaussian   0.3              0.330303  0.019192   2.815163   
6     14.0      real   NaN              0.541414  0.019192   7.856494   
7     14.0  gaussian   0.3              0.368687  0.019192   3.348571   
8     18.0      real   NaN              0.579798  0.019192   9.189829   
9     18.0  gaussian   0.3              0.387879  0.019192   3.620568   
10    22.0      real   NaN              0.618182  0.019192  10.682352   
11    22.0  gaussian   0.3              0.407071  0.019192   3.923020   
12    26.0      real   NaN              0.637374  0.019192  11.295762   
13    26.0  gaussian   0.3              0.426263  0.019192   4.243256   
14    30.0      real   NaN              0.637374  0.019192  10.953281   
15    30.0  gaussian   0.3              0.464646  0.019192   5.030415   

                                                     
     rho err          N0    N0 err lambda      Chi2  
0   0.001831    4.485465  0.210999    0.5  0.000110  
1   0.000420  104.759663  1.681834    0.5  0.000057  
2   0.002475    0.244450  0.006613    0.5  0.000026  
3   0.001489   14.358209  0.423139    0.5  0.000103  
4   0.003990    0.023326  0.000526    0.5  0.000015  
5   0.002020    3.154664  0.078732    0.5  0.000053  
6   0.009307    0.013519  0.000513    0.5  0.000047  
7   0.001954    1.746794  0.033551    0.5  0.000030  
8   0.014074    0.011185  0.000519    0.5  0.000077  
9   0.001477    1.518145  0.019630    0.5  0.000014  
10  0.029054    0.010497  0.000816    0.5  0.000238  
11  0.001533    1.252694  0.015045    0.5  0.000012  
12  0.034680    0.014639  0.001222    0.5  0.000313  
13  0.002111    1.075317  0.015910    0.5  0.000020  
14  0.034471    0.025527  0.002111    0.5  0.000354  
15  0.004125    0.625912  0.014663    0.5  0.000049

# Fokker-Planck Fittings
## Fixed $\kappa$, exploring $I_\ast$

In [25]:
labels = (
    ("epsilon", ""),
    ("type", ""),
    ("sigma", ""),
    ("Fokker-Planck", "I_star"),
    ("Fokker-Planck", "time_scale"),
    ("Fit data", "error"),
)

fp_data = []
fp_fit_data = pd.DataFrame(columns=pd.MultiIndex.from_tuples(labels))

In [26]:
for i in tqdm(range(len(fitting_data[("Model 4 (3 free pars)", "k")]))):
    # Setup initial conditions
    I_max = cut_point**2 / 2
    I = np.linspace(0, I_max, 1000)
    I0 = I * np.exp(-(I/sigma**2))
    I0 /= integrate.trapz(I0, I)
    # Execute research
    values, I_star, t, error = fit.scan_I_star(
        lost_table[i],
        turn_sampling,
        I_max,
        I0,
        fitting_data[("Model 4 (3 free pars)", "k")][i],
        0.2,
        0.01,
        10,
        0.05
    )
    # Save data
    fp_data.append(values)
    fp_fit_data.loc[len(fp_fit_data)] = [
        fitting_data[("epsilon", "")][i],
        fitting_data[("type", "")][i],
        fitting_data[("sigma", "")][i],
        I_star,
        t,
        error
    ]

k=0.23434343434343433, I_star=0.19, dt=0.01
k=0.23434343434343433, I_star=0.2, dt=0.01
k=0.23434343434343433, I_star=0.21000000000000002, dt=0.01
Going UP!
k=0.23434343434343433, I_star=0.22050000000000003, dt=0.01
k=0.23434343434343433, I_star=0.23152500000000004, dt=0.01
k=0.23434343434343433, I_star=0.24310125000000005, dt=0.01
k=0.23434343434343433, I_star=0.2552563125000001, dt=0.01
k=0.23434343434343433, I_star=0.2680191281250001, dt=0.01
k=0.23434343434343433, I_star=0.2814200845312501, dt=0.01
k=0.23434343434343433, I_star=0.29549108875781266, dt=0.01
increase!
k=0.23434343434343433, I_star=0.3102656431957033, dt=0.1
k=0.23434343434343433, I_star=0.3257789253554885, dt=0.1
k=0.1575757575757576, I_star=0.19, dt=0.01
k=0.1575757575757576, I_star=0.2, dt=0.01
k=0.1575757575757576, I_star=0.21000000000000002, dt=0.01
Going UP!
k=0.1575757575757576, I_star=0.22050000000000003, dt=0.01
k=0.1575757575757576, I_star=0.23152500000000004, dt=0.01
k=0.3686868686868686, I_star=0.19, dt=0.0

In [27]:
fp_fit_data

epsilon      type sigma Fokker-Planck  \
                                  I_star   
0      2.0      real   NaN      0.310266   
1      2.0  gaussian   0.3      0.220500   
2      6.0      real   NaN      0.585052   
3      6.0  gaussian   0.3      0.310266   
4     10.0      real   NaN      1.103203   
5     10.0  gaussian   0.3      0.436575   
6     14.0      real   NaN      1.407998   
7     14.0  gaussian   0.3      0.530660   
8     18.0      real   NaN      1.552318   
9     18.0  gaussian   0.3      0.557193   
10    22.0      real   NaN      1.886852   
11    22.0  gaussian   0.3      0.614305   
12    26.0      real   NaN      2.080254   
13    26.0  gaussian   0.3      0.677271   
14    30.0      real   NaN      1.886852   
15    30.0  gaussian   0.3      0.784026   

                                                       Fit data  
                                           time_scale     error  
0   [436.0, 435.0, 434.0, 433.0, 432.0, 431.0, 430...  0.009252  
1   [22.900000000000002, 22.8, 22.7, 22.6, 22.5, 2...  0.004905  
2   [13540.0, 13530.0, 13520.0, 13510.0, 13500.0, ...  0.010820  
3   [95.5, 95.4, 95.3, 95.2, 95.10000000000001, 95...  0.012351  
4   [410000.0, 409000.0, 408000.0, 407000.0, 40600...  0.012547  
5   [591.0, 590.0, 589.0, 588.0, 587.0, 586.0, 585...  0.013779  
6   [474000.0, 473000.0, 472000.0, 471000.0, 47000...  0.011806  
7   [2096.0, 2095.0, 2094.0, 2093.0, 2092.0, 2091....  0.012824  
8   [211800.0, 211700.0, 211600.0, 211500.0, 21140...  0.011484  
9   [1736.0, 1735.0, 1734.0, 1733.0, 1732.0, 1731....  0.015139  
10  [416000.0, 415000.0, 414000.0, 413000.0, 41200...  0.011968  
11  [3330.0, 3320.0, 3310.0, 3300.0, 3290.0, 3280....  0.014523  
12  [673000.0, 672000.0, 671000.0, 670000.0, 66900...  0.013273  
13  [6200.0, 6190.0, 6180.0, 6170.0, 6160.0, 6150....  0.016143  
14  [242000.0, 241000.0, 240000.0, 239000.0, 23800...  0.010115  
15  [7040.0, 7030.0, 7020.0, 7010.0, 7000.0, 6990....  0.015665

In [28]:
plt.figure()

plt.plot(
    fp_fit_data[fp_fit_data[("type")] == "gaussian"][("epsilon")],
    fp_fit_data[fp_fit_data[("type")] == "gaussian"][("Fokker-Planck", "I_star")]
)
plt.xlabel("$\\varepsilon$")
plt.ylabel("$I_\\ast$")
plt.title("Best value for $I_\\ast$")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Best value for $I_\\ast$')

## Double FP fit for both $\kappa$ and $I_\ast$

In [45]:
labels = (
    ("epsilon", ""),
    ("type", ""),
    ("sigma", ""),
    ("Fokker-Planck", "k"),
    ("Fokker-Planck", "I_star"),
    ("Fokker-Planck", "time_scale"),
    ("Fit data", "error")
)

fp_double_data = []
fp_double_fit_data = pd.DataFrame(columns=pd.MultiIndex.from_tuples(labels))

In [ ]:
for i in tqdm(range(len(fitting_data[("Model 4 (3 free pars)", "k")]))):
    # Setup initial conditions
    I_max = cut_point**2 / 2
    I = np.linspace(0, I_max, 1000)
    I0 = I * np.exp(-(I/sigma**2))
    I0 /= integrate.trapz(I0, I)
    # Execute research
    values, k, I_star, t, error = fit.scan_k(
        lost_table[i],
        turn_sampling,
        I_max,
        I0,
        fitting_data[("Model 4 (3 free pars)", "k")][i],
        0.2,
        0.001,
        10,
        0.05,
        0.05
    )
    fp_double_data.append(values)
    fp_double_fit_data.loc[len(fp_double_fit_data)] = [
        fitting_data[("epsilon", "")][i],
        fitting_data[("type", "")][i],
        fitting_data[("sigma", "")][i],
        k,
        I_star,
        t,
        error,
    ]

k=0.2226262626262626, I_star=0.19, dt=0.001
k=0.2226262626262626, I_star=0.2, dt=0.001
k=0.2226262626262626, I_star=0.21000000000000002, dt=0.001
Going UP!
k=0.2226262626262626, I_star=0.22050000000000003, dt=0.001
k=0.2226262626262626, I_star=0.23152500000000004, dt=0.001
k=0.2226262626262626, I_star=0.24310125000000005, dt=0.001
k=0.2226262626262626, I_star=0.2552563125000001, dt=0.001
increase!
k=0.2226262626262626, I_star=0.2680191281250001, dt=0.01
k=0.2226262626262626, I_star=0.2814200845312501, dt=0.01
k=0.2226262626262626, I_star=0.29549108875781266, dt=0.01
k=0.23434343434343433, I_star=0.19, dt=0.001
k=0.23434343434343433, I_star=0.2, dt=0.001
k=0.23434343434343433, I_star=0.21000000000000002, dt=0.001
Going UP!
k=0.23434343434343433, I_star=0.22050000000000003, dt=0.001
k=0.23434343434343433, I_star=0.23152500000000004, dt=0.001
k=0.23434343434343433, I_star=0.24310125000000005, dt=0.001
k=0.23434343434343433, I_star=0.2552563125000001, dt=0.001
k=0.23434343434343433, I_star

In [44]:
i = 4

plt.figure()
plt.plot(turn_sampling, fp_double_data[i][:-1])
plt.plot(turn_sampling, lost_table[i])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [34]:
fp_double_fit_data

epsilon      type sigma Fokker-Planck            \
                                       k    I_star   
0      2.0      real   NaN      0.200920  0.255256   
1      2.0  gaussian   0.3      0.157576  0.220500   
2      6.0      real   NaN      0.368687  0.585052   
3      6.0  gaussian   0.3      0.196181  0.243101   
4     10.0      real   NaN      0.414831  0.711135   
5     10.0  gaussian   0.3      0.298098  0.377130   
6     14.0      real   NaN      0.440985  0.746691   
7     14.0  gaussian   0.3      0.316103  0.395986   
8     18.0      real   NaN      0.497104  0.952988   
9     18.0  gaussian   0.3      0.350061  0.458404   
10    22.0      real   NaN      0.587273  1.552318   
11    22.0  gaussian   0.3      0.386717  0.557193   
12    26.0      real   NaN      0.519145  1.000638   
13    26.0  gaussian   0.3      0.329834  0.395986   
14    30.0      real   NaN      0.637374  1.886852   
15    30.0  gaussian   0.3      0.464646  0.784026   

                                                       Fit data  
                                           time_scale     error  
0   [55.6, 55.5, 55.4, 55.300000000000004, 55.2, 5...  0.005488  
1   [22.75, 22.740000000000002, 22.73, 22.72, 22.7...  0.005109  
2   [13540.0, 13530.0, 13520.0, 13510.0, 13500.0, ...  0.010820  
3   [29.560000000000002, 29.55, 29.54, 29.53, 29.5...  0.006097  
4   [27200.0, 27100.0, 27000.0, 26900.0, 26800.0, ...  0.010374  
5   [345.0, 344.0, 343.0, 342.0, 341.0, 340.0, 339...  0.009693  
6   [13770.0, 13760.0, 13750.0, 13740.0, 13730.0, ...  0.010102  
7   [350.0, 349.0, 348.0, 347.0, 346.0, 345.0, 344...  0.010344  
8   [26800.0, 26700.0, 26600.0, 26500.0, 26400.0, ...  0.009599  
9   [693.0, 692.0, 691.0, 690.0, 689.0, 688.0, 687...  0.012143  
10  [170800.0, 170700.0, 170600.0, 170500.0, 17040...  0.011582  
11  [2460.0, 2450.0, 2440.0, 2430.0, 2420.0, 2410....  0.012934  
12  [25400.0, 25300.0, 25200.0, 25100.0, 25000.0, ...  0.009699  
13  [318.0, 317.0, 316.0, 315.0, 314.0, 313.0, 312...  0.009632  
14  [242000.0, 241000.0, 240000.0, 239000.0, 23800...  0.010115  
15  [7040.0, 7030.0, 7020.0, 7010.0, 7000.0, 6990....  0.015665

In [32]:
plt.figure()
plt.scatter(
    fp_double_fit_data[fp_double_fit_data["type"] == "gaussian"]["epsilon"],
    fp_double_fit_data[fp_double_fit_data["type"] == "gaussian"][("Fokker-Planck", "k")],
    label="$\\kappa$, FP best fit"
)
plt.scatter(
    fitting_data[fitting_data["type"] == "gaussian"]["epsilon"],
    fitting_data[fitting_data["type"] == "gaussian"][("Model 4 (3 free pars)", "k")],
    label="$\\kappa$, Model 4 best fit"
)

plt.legend()
plt.xlabel("$\\varepsilon$")
plt.ylabel("$\\kappa$")
plt.title("Comparison between Model 4 best fit and Fokker-Planck result")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Comparison between Model 4 best fit and Fokker-Planck result')

In [33]:
plt.figure()
plt.scatter(
    fp_double_fit_data[fp_double_fit_data["type"] == "gaussian"]["epsilon"],
    fp_double_fit_data[fp_double_fit_data["type"] == "gaussian"][("Fokker-Planck", "I_star")],
    label="$I_\\ast$, FP best fit"
)

plt.legend()
plt.xlabel("$\\varepsilon$")
plt.ylabel("$I_\\ast$")
plt.title("Comparison between Model 4 best fit and Fokker-Planck result")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Comparison between Model 4 best fit and Fokker-Planck result')